# Informe: Diferenciación e Integración numérica
Este notebook ejecuta los ejemplos del proyecto, genera tablas con las aproximaciones
y grafica errores para estudiar la convergencia. Está diseñado para ser ejecutado
en Windows o Linux usando el entorno virtual creado por `bootstrap_env.py`.

## 1) Preparación del entorno y módulos
Asegúrate de haber ejecutado `python bootstrap_env.py` desde la carpeta `Metodos de aproximacion`
para crear el entorno y las dependencias. Luego activa el entorno y ejecuta las celdas siguientes.

In [ ]:
# Importar módulos principales y preparar paths (funciona con carpeta con espacios)
import os
import sys
# Añadir la carpeta del módulo al path
module_dir = os.path.abspath(os.path.dirname(__file__)) if '__file__' in globals() else os.getcwd()
module_dir = os.path.join(module_dir)
if module_dir not in sys.path:
    sys.path.insert(0, module_dir)
from numerical_tools import (
    f_x_ln_x, f_1_plus_ln_x, exact_derivative_x_ln_x, exact_integral_of_1_plus_ln_x,
    forward_difference, backward_difference, central_difference, three_point_forward, three_point_backward,
    midpoint_rule, trapezoid_rule, simpson_rule, composite_simpson, composite_midpoint, max_abs_on_interval
)
print('Módulos cargados con éxito')

## 2) Diferenciación — f(x)=x ln x en x0=2 con h=0.1
Calculamos las aproximaciones solicitadas y mostramos una tabla. También estimamos las cotas usando muestreo.

In [ ]:
x0 = 2.0
h = 0.1
a = x0 - 2*h
b = x0 + 2*h
methods = {
    'Forward (1-step)': forward_difference(f_x_ln_x, x0, h),
    'Backward (1-step)': backward_difference(f_x_ln_x, x0, h),
    'Central (3-pt)': central_difference(f_x_ln_x, x0, h),
    '3-pt forward (left)': three_point_forward(f_x_ln_x, x0, h),
    '3-pt backward (right)': three_point_backward(f_x_ln_x, x0, h),
}
exact = exact_derivative_x_ln_x(x0)
rows = []
for name, value in methods.items():
    # estimación simple de cota
    if 'Central' in name:
        def f3(x):
            return -1.0/(x*x)
        M = max_abs_on_interval(lambda x: abs(f3(x)), a, b)
        cota = M*(h**2)/6.0
    elif '3-pt' in name:
        def f3(x):
            return -1.0/(x*x)
        M = max_abs_on_interval(lambda x: abs(f3(x)), a, b)
        cota = M*(h**2)/3.0
    else:
        def f2(x):
            return 1.0/x
        M = max_abs_on_interval(lambda x: abs(f2(x)), a, b)
        cota = M*abs(h)/2.0
    rows.append({
        'Metodo': name,
        'Aproximacion': value,
        'Valor exacto': exact,
        'Error absoluto': abs(value-exact),
        'Cota teorica (estimada)': cota,
    })

# Mostrar con pandas si está disponible
try:
    import pandas as pd
    df = pd.DataFrame(rows)
    display(df)
except Exception:
    for r in rows:
        print(r)

## 3) Integración — f(x)=1+ln x en [1,2]
Calculamos las reglas solicitadas, comparamos con la integral exacta y mostramos cotas estimadas.

In [ ]:
a = 1.0
b = 2.0
results = []
I_mid = midpoint_rule(f_1_plus_ln_x, a, b)
I_trap = trapezoid_rule(f_1_plus_ln_x, a, b)
I_simp = simpson_rule(f_1_plus_ln_x, a, b)
I_simp_comp = composite_simpson(f_1_plus_ln_x, a, b, m=1)
I_mid_comp = composite_midpoint(f_1_plus_ln_x, a, b, n=5)
I_exact = exact_integral_of_1_plus_ln_x(a, b)
# estimaciones simples de las cotas
def f2(x):
    return -1.0/(x*x)
M2 = max_abs_on_interval(lambda x: abs(f2(x)), a, b)
cota_mid = ((b-a)**3)/24.0 * M2
cota_trap = ((b-a)**3)/12.0 * M2
# Para Simpson usamos una estimación de f4
def f4(x):
    return 6.0/(x**4)
M4 = max_abs_on_interval(lambda x: abs(f4(x)), a, b)
cota_simp = ((b-a)**5)/2880.0 * M4
h = (b-a)/5.0
cota_mid_comp = 5 * ((h**3)/24.0) * M2
results = [
    ('Midpoint (simple)', I_mid, cota_mid),
    ('Trapezoid (simple)', I_trap, cota_trap),
    ('Simpson (simple)', I_simp, cota_simp),
    ('Simpson compuesta (2 subintervalos)', I_simp_comp, cota_simp),
    ('Midpoint compuesta (5 subintervalos)', I_mid_comp, cota_mid_comp),
]
rows = []
for name, val, cota in results:
    rows.append({
        'Metodo': name,
        'Aproximacion': val,
        'Valor exacto': I_exact,
        'Error absoluto': abs(val - I_exact),
        'Cota teorica (estimada)': cota,
    })
try:
    import pandas as pd
    df = pd.DataFrame(rows)
    display(df)
except Exception:
    for r in rows:
        print(r)

## 4) Gráficas de convergencia (opcional)
Aquí vemos cómo cambia el error con h para las fórmulas más comunes.

In [ ]:
import math
try:
    import matplotlib.pyplot as plt
except Exception:
    plt = None

# Barrido de h para estudiar convergencia en x0=2
x0 = 2.0
hs = [10**(-k) for k in range(1,6)]
errs_central = []
errs_forward = []
for h in hs:
    cen = central_difference(f_x_ln_x, x0, h)
    fwd = forward_difference(f_x_ln_x, x0, h)
    exact = exact_derivative_x_ln_x(x0)
    errs_central.append(abs(cen - exact))
    errs_forward.append(abs(fwd - exact))

if plt is not None:
    plt.loglog(hs, errs_forward, '-o', label='Forward')
    plt.loglog(hs, errs_central, '-o', label='Central')
    plt.gca().invert_xaxis()
    plt.xlabel('h')
    plt.ylabel('Error absoluto')
    plt.legend()
    plt.title('Convergencia de fórmulas de derivada')
    plt.show()
else:
    print('matplotlib no está disponible: mostrando tabla de errores')
    for k, h in enumerate(hs):
        print(f'h={h}: forward={errs_forward[k]:.3e}, central={errs_central[k]:.3e}')

---
### Notas finales
- Si planeas ejecutar en Linux, usa `python3 bootstrap_env.py` y luego `source .venv/bin/activate`.
- El notebook intenta usar `pandas` y `matplotlib` si están disponibles; si no, imprime tablas y valores por consola.
- Para automatizar completamente (por ejemplo en entornos CI), ejecuta `bootstrap_env.py` y luego `python run_differentiation.py --no-pandas` o `python run_integration.py --no-pandas`.